In [ ]:
from datasets import load_dataset
dataset = load_dataset('Babelscape/rebel-dataset')

Generating train split:   0%|          | 0/3120296 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/172860 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/173601 [00:00<?, ? examples/s]

Dataset rebel-dataset downloaded and prepared to /Users/wfh/.cache/huggingface/datasets/Babelscape___rebel-dataset/REBEL/1.0.0/3c1f1de369845ba9f822bc11f2d1923231b4d43fd79780e79ed0aa5843504fbd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'triplets'],
    num_rows: 3120296
})

In [18]:
from langsmith import Client

client = Client()

In [ ]:
?client.upload_dataframe

Signature:
client.upload_dataframe(
    df: 'pd.DataFrame',
    name: 'str',
    input_keys: 'Sequence[str]',
    output_keys: 'Sequence[str]',
    *,
    description: 'Optional[str]' = None,
    data_type: 'Optional[DataType]' = <DataType.kv: 'kv'>,
) -> 'Dataset'
Docstring:
Upload a dataframe as individual examples to the LangSmith API.

Parameters
----------
df : pd.DataFrame
    The dataframe to upload.
name : str
    The name of the dataset.
input_keys : Sequence[str]
    The input keys.
output_keys : Sequence[str]
    The output keys.
description : str or None, default=None
    The description of the dataset.
data_type : DataType or None, default=DataType.kv
    The data type of the dataset.

Returns
-------
Dataset
    The uploaded dataset.

Raises
------
ValueError
    If the csv_file is not a string or tuple.
File:      ~/code/lc/langchainplus-sdk/python/langsmith/client.py
Type:      method

In [37]:
client.upload_dataframe(df, "test-foo", input_keys=["context"], output_keys=["triplets"])

Dataset(name='test-foo', description=None, data_type=<DataType.kv: 'kv'>, id=UUID('b9ebf406-8fe1-4230-8fe1-b2fd87d02ba8'), created_at=datetime.datetime(2023, 8, 18, 20, 0, 15, 241535), modified_at=None)

In [36]:
df = pd.DataFrame(train[:1000]).sample(100)

In [40]:
import pandas as pd
# train, dev, test = dataset['train'], dataset['validation'], dataset['test']

def sample_and_upload(dataset, split, client, sample_size: int = 100):
    dataset_name = f"Rebel-linearized-{split}"
    sampled = pd.DataFrame(dataset[split][:2000]).sample(sample_size)
    ds = client.upload_dataframe(
                    sampled,
                    dataset_name,
                    input_keys=["context"],
                    output_keys=["triplets"],
        )
    return ds
    

In [48]:
import pandas as pd

for split in ['train', 'validation', 'test']:
    name = f"Rebel-linearized-{split}"
    examples = [{**x.inputs, **x.outputs} for x in client.list_examples(dataset_name=name)]
    pd.DataFrame(examples).to_csv(f"data/{name}.csv", index=False)

In [43]:
ds = sample_and_upload(dataset, 'test', client)

In [ ]:
dataset['test'][0:5]['triplets']

['<triplet> Trinity Peninsula <subj> Graham Land <obj> part of <subj> Antarctica <obj> continent <triplet> Graham Land <subj> Antarctica <obj> continent',
 '<triplet> Trinity Peninsula <subj> Graham Land <obj> part of <triplet> Graham Land <subj> Antarctica <obj> continent',
 '<triplet> Harald Kaas <subj> 19 May 1868 <obj> date of birth <subj> 5 December 1953 <obj> date of death',
 '<triplet> Polytechnicum <subj> Munich <obj> headquarters location',
 '<triplet> Bergen Line <subj> Norwegian State Railways <obj> owned by']

In [14]:
transform_to_json(dataset['test'][0]['triplets'])

[{'triplet': {'subj': 'Trinity',
   'obj': '<subj> Antarctica',
   'pred': 'Graham part continent'}},
 {'triplet': {'subj': 'Graham',
   'obj': '<subj>',
   'pred': 'Antarctica continent'}}]